<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1">Обзор данных</a></span></li><li><span><a href="#Задача-1" data-toc-modified-id="Задача-1-2">Задача 1</a></span></li><li><span><a href="#Задача-2" data-toc-modified-id="Задача-2-3">Задача 2</a></span></li><li><span><a href="#Задача-3" data-toc-modified-id="Задача-3-4">Задача 3</a></span></li><li><span><a href="#Задача-4" data-toc-modified-id="Задача-4-5">Задача 4</a></span></li><li><span><a href="#Задача-5" data-toc-modified-id="Задача-5-6">Задача 5</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-7">Выводы</a></span></li></ul></div>

# Исследование базы данных книжного сервиса

Компания купила крупный сервис для чтения книг по подписке. Необходимо проанализировать базу данных, содержащих информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг.

**Цель исследования** - получить данные для формирования ценностного предложения для нового продукта.

**Задачи проекта:**
1. Найти количество книг, вышедших после 1 января 2000 года.
2. Найти количество обзоров и среднюю оценку для каждой книги.
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц.
4. Определить автора с самой высокой средней оценкой книг.
5. Вычислить среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**Источник данных** — база данных сервиса для чтения книг.

## Обзор данных

Загрузим библиотеки, необходимые для работы:

In [1]:
import pandas as pd
from sqlalchemy import create_engine

Теперь подключимся к базе данных:

In [2]:
db_config = {
    'user': '',
    'pwd': '',
    'host': '',
    'port': 6432,
    'db': 'data-analyst-final-project-db'
}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'], db_config['pwd'], db_config['host'], db_config['port'],
    db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode': 'require'})

Изучим таблицы. Откроем первые строки и посчитаем количество записей для каждой из них. Начнём с таблицы `books`.

In [3]:
query = '''
            SELECT * FROM books
            LIMIT 5
        '''
display(pd.io.sql.read_sql(query, con=engine))

query = '''
            SELECT COUNT(book_id) AS books_total
            FROM books
        '''
pd.io.sql.read_sql(query, con=engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,books_total
0,1000


Из условия известно:

`books` — таблица книг:
- `book_id` — идентификатор книги (первичный ключ),
- `author_id` — идентификатор автора (внешний ключ),
- `title` — название книги,
- `num_pages` — количество страниц,
- `publication_date` — дата публикации книги,
- `publisher_id` — идентификатор издателя (внешний ключ).


Таблица содержит данные о 1000 книгах. Первичным ключом таблицы `books` является идентификатор книги — `book_id`. В таблице также есть два внешних ключа — `author_id` для связи с таблицей `authors` и `publisher_id` для связи с таблицей `publishers`.

Выведем те же самые данные для таблицы `authors`:

In [4]:
query = '''
            SELECT * FROM authors
            LIMIT 5
        '''
display(pd.io.sql.read_sql(query, con=engine))

query = '''
            SELECT COUNT(author_id) AS authors_total
            FROM authors
        '''
pd.io.sql.read_sql(query, con=engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,authors_total
0,636


Структура таблицы:

`authors` — таблица авторов:

- `author_id` — идентификатор автора (первичный ключ),
- `author` — имя автора.

В данной таблице представлена информация о 636 авторах. В таблице всего два столбца - `author_id` (первичный ключ) и имя автора.

Перейдём к таблице `publishers`:

In [5]:
query = '''
            SELECT * FROM publishers
            LIMIT 5
        '''
display(pd.io.sql.read_sql(query, con=engine))

query = '''
            SELECT COUNT(publisher_id) AS publishers_total
            FROM publishers
        '''
pd.io.sql.read_sql(query, con=engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,publishers_total
0,340


Структура таблицы:

`publishers` — таблица издательств:

- `publisher_id` — идентификатор издательства (первичный ключ),
- `publisher` — название издательства.

В таблице состоит из двух столбцов - id издательства (первичный ключ) и название издательства. В ней содержится 340 записей.

Перейдём к таблице `ratings`. Сразу же выведем минимальную и максимальную оценки:

In [6]:
query = '''
            SELECT * FROM ratings
            LIMIT 5
        '''
display(pd.io.sql.read_sql(query, con=engine))

query = '''
            SELECT COUNT(rating_id) AS ratings_total
            FROM ratings
        '''
display(pd.io.sql.read_sql(query, con=engine))

query = '''
            SELECT MIN(rating),
                   MAX(rating)
            FROM ratings
        '''

pd.io.sql.read_sql(query, con=engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,ratings_total
0,6456


,min,max
0,1,5


Из условия:

`ratings` — таблица пользовательских оценок книг:
- `rating_id` — идентификатор оценки (первичный ключ),
- `book_id` — идентификатор книги (внешний ключ),
- `username` — имя пользователя, оставившего оценку,
- `rating` — оценка книги.

Пользователи сервиса поставили 6456 оценок в диапозоне от 1 до 5. В таблице содержатся первичный ключ `rating_id` и внешний ключ `book_id` для связи с таблицей `books`.

Наконец, изучим таблицу `reviews`:

In [7]:
query = '''
            SELECT * FROM reviews
            LIMIT 5
        '''

display(pd.io.sql.read_sql(query, con=engine))

query = '''
            SELECT COUNT(review_id) AS reviews_total
            FROM reviews
        '''

pd.io.sql.read_sql(query, con=engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


,reviews_total
0,2793


`reviews` — таблица пользовательских обзоров на книги:
- `review_id` — идентификатор обзора (первичный ключ),
- `book_id` — идентификатор книги (внешний ключ),
- `username` — имя пользователя, написавшего обзор,
- `text` — текст обзора.

В таблице представлены данные о 2793 обзорах пользователей. В ней также присуствуют первичный (`review_id`) и внешний (`book_id`) ключи.

**Промежуточные вывод:** сервис предлагает 1000 книг от 636 разных авторов, которые были опубликованы 340 издательствами; пользователи сервиса выставили 6456 оценок и написали 2793 обзора.

## Задача 1

Посчитаем, сколько книг вышло после 1 января 2000 г.:

In [8]:
query = '''
            SELECT COUNT(book_id) AS books_of_21_century
            FROM books
            WHERE CAST(publication_date AS timestamp) >= '2000-01-01'
        '''

pd.io.sql.read_sql(query, con=engine)

,books_of_21_century
0,821


Больше 80% представленных в сервисе книг вышло после 1 января 2000 г. Иными словами, в сервисе в большинстве своём представлена современная литература.

## Задача 2

Для каждой книги найдём количество обзоров и среднюю оценку. Выведем на экран топ-10 книг по количеству обзоров и среднему рейтингу:

In [7]:
query = '''
            WITH rev_cnt AS
            (SELECT book_id,
                    COUNT(review_id) AS reviews_cnt
            FROM reviews
            GROUP BY book_id),
            
            rev_avg AS 
            (SELECT book_id,
                    AVG(rating) AS avg_rating
            FROM ratings
            GROUP BY book_id)
            
            
            
            SELECT b.book_id,
                   b.title,
                   rev_cnt.reviews_cnt,
                   rev_avg.avg_rating
            FROM books AS b
            LEFT JOIN rev_cnt ON rev_cnt.book_id = b.book_id
            JOIN rev_avg ON rev_avg.book_id = b.book_id
            ORDER BY rev_avg.avg_rating DESC, rev_cnt.reviews_cnt DESC 
        '''

pd.io.sql.read_sql(query, con=engine)

,book_id,title,reviews_cnt,avg_rating
0,672,The Cat in the Hat and Other Dr. Seuss Favorites,NaN,5.00
1,17,A Dirty Job (Grim Reaper #1),4.0,5.00
2,553,School's Out—Forever (Maximum Ride #2),3.0,5.00
3,444,Moneyball: The Art of Winning an Unfair Game,3.0,5.00
4,516,Plum Island (John Corey #1),2.0,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3.0,2.25
996,202,Drowning Ruth,3.0,2.00
997,371,Junky,2.0,2.00
998,316,His Excellency: George Washington,2.0,2.00


По таблице не видно сильной зависимости между рейтингом и количеством отзывов. Высокие рейтинги имеют как книги с большим количеством отзывов, так и без отзывов вообще. Хотя вот самые низкие рейтинги имеют всё же книги с отзывами.

## Задача 3

Найдём издательство, которое выпустило наибольшее число книг толще 50 страниц (чтобы исключить брошюры):

In [10]:
query = '''
            SELECT p.publisher,
                   COUNT(b.book_id) AS books_total
            FROM books AS b
            JOIN publishers AS p ON b.publisher_id = p.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher_id
            ORDER BY books_total DESC
            LIMIT 1
        '''

pd.io.sql.read_sql(query, con=engine)

,publisher,books_total
0,Penguin Books,42


Наибольшее количество книг (42), представленных в сервисе, выпущено издательством Penguin Books.

## Задача 4

Определим автора с самой высокой средней оценкой книг:

In [16]:
query = '''
            SELECT a.author,
                   AVG(r.rating) AS avg_rating
            FROM authors AS a
            JOIN books AS b ON a.author_id = b.author_id
            JOIN ratings AS r ON b.book_id = r.book_id
            WHERE r.book_id IN (SELECT book_id
                                FROM ratings
                                GROUP BY book_id
                                HAVING COUNT(rating_id) >= 50)
            GROUP BY a.author_id
            ORDER BY avg_rating DESC
            LIMIT 1
        '''

pd.io.sql.read_sql(query, con=engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


Самыми популярными авторами оказались Джоан Роулинг в соавторстве с Мэри Гранпре, создавшими серию книг о Гарри Поттере. Им пренадлежит самая высокая средняя оценка пользователей — 4.3.

## Задача 5

Посчитаем среднее количество обзоров от пользователей, которые поставили более 50 оценок:

In [12]:
query = '''
            SELECT AVG(reviews_cnt) AS avg_reviews FROM (
                                                             SELECT DISTINCT username,
                                                                    COUNT(review_id) AS reviews_cnt
                                                             FROM reviews
                                                             WHERE username IN (SELECT username
                                                                                FROM ratings
                                                                                GROUP BY username
                                                                                HAVING COUNT(rating_id) >= 50) 
                                                             GROUP BY username) AS re
        '''

pd.io.sql.read_sql(query, con=engine)

,avg_reviews
0,24.222222


В среднем пользователи, которые поставили более 50 оценок, пишут около 24 обзоров.

## Выводы

1. В сервисе представлены 1000 книг, написанных 636 авторами и опубликованных 340 издательствами. Пользователи сервиса поставили 6456 оценок от 1 до 5, а также написали 2793 обзора.

2. Более 80% книг, представленных в сервисе, изданы в XXI веке.

3.  Сильной зависимости между рейтингом и количеством отзывов не наблюдается. Высокие рейтинги имеют как книги с большим количеством отзывов, так и без отзывов вообще. Самые низкие рейтинги имеют всё же книги с отзывами.

4. Больше всего книг, доступных в серсиве, было выпущено издательством Penguin Books.

5. Самая высокая средняя оценка пользователей в сервисе принадлежит книгам Джоан Роулинг совместно с Мари Гранпре.

6. Пользователи, которые ставят больше 50 оценок, в среднем пишут около 24 рецензий.